<a href="https://colab.research.google.com/github/AliceRayLu/CS246-Winter24/blob/main/CS246_Colab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 1
## Word Count in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive2
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can comment out the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out
!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
# !sudo ldconfig
#If error related to the above execution occurs, you can try commenting out the above 12 lines under pip install PyDrive2 (not included)
!sudo apt-get upgrade --fix-missing
!apt install openjdk-11-jdk-headless
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can comment out the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=114c02d23ab102487de00d890b3788e0ecf7857836f723bf22dc5f5aeac19d87
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Calculating upgrade... Done
The following packages have been kept back:
  libcudnn8 libcudnn8-dev libnccl-dev libnccl2
The following packages will be upgraded:
  base-files bash binutils binutils-common binutils-x86-64-linux-gnu bsdutils
  coreutils cuda-compat-12-2 cuda-keyring cuda-toolkit-12-config-common
  cuda-toolkit-config-common dpkg dpkg-dev libbinutils libblkid1 libc-bin
  libctf-nobfd0 libctf0 

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you successfully run the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter, we want to count the total number of (non-unique) words that start with a specific letter. (If a specific (aka unique) word that starts with letter 'a' appears N times, it should be counted in words starting with 'a' N times.)

In your implementation, **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all words that **start** with a non-alphabetic character. You should output word counts for the **entire document**, inclusive of the title, author, and the main texts. If you encounter words broken as a result of new lines, e.g. "pro-ject" where the segment after the dash sign is on a new line, no special processing is needed and you can safely consider it as two words ("pro" and "ject").

Your outputs will be graded on a range -- if your differences from the ground-truths are within an error threshold of 5, you'll be considered correct.

**Hint:**
1. split only on space (' ') but not hyphen/dash ('-') or other symbols.
2. you may find spark functions explode (https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.explode.html) and split (https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.functions.split.html) helpful, but you don't need to restrict to them as long as you can satisfy your goal.

Clarification:

1. If a word 'project' is separated into two lines in the form of 'pro-' in the first line and 'ject' in the second line, it should be treated as two words (when you import the text using spark.read.text, it treats each newline as a new row in the DataFrame). However, for the word 'self-love' that appears in a single line, it should be treated as one word starting with letter 's'.



In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [6]:
# YOUR CODE HERE
txt = spark.read.text("./pg100.txt")
txt.show(3)

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
| William Shakespeare|
|                    |
+--------------------+
only showing top 3 rows



Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!

In [7]:
splitted = txt.select(explode(split(txt.value, " ")).alias("word"))
splitted.show()

+------------+
|        word|
+------------+
|         The|
|     Project|
|   Gutenberg|
|       EBook|
|          of|
|         The|
|    Complete|
|       Works|
|          of|
|     William|
|Shakespeare,|
|          by|
|     William|
| Shakespeare|
|            |
|        This|
|       eBook|
|          is|
|         for|
|         the|
+------------+
only showing top 20 rows



In [14]:
# change all words into lower case
lowered = splitted.select(lower(splitted.word).alias("word"))

# filter empty word
lowered = lowered.filter(lowered.word != "")
lowered.show()

+------------+
|        word|
+------------+
|         the|
|     project|
|   gutenberg|
|       ebook|
|          of|
|         the|
|    complete|
|       works|
|          of|
|     william|
|shakespeare,|
|          by|
|     william|
| shakespeare|
|        this|
|       ebook|
|          is|
|         for|
|         the|
|         use|
+------------+
only showing top 20 rows



In [15]:
# add a new column which displays the first letter
first_letter = lowered.select(lowered.word, substring(lowered.word, 1, 1).alias("first_letter"))
first_letter.show()

+------------+------------+
|        word|first_letter|
+------------+------------+
|         the|           t|
|     project|           p|
|   gutenberg|           g|
|       ebook|           e|
|          of|           o|
|         the|           t|
|    complete|           c|
|       works|           w|
|          of|           o|
|     william|           w|
|shakespeare,|           s|
|          by|           b|
|     william|           w|
| shakespeare|           s|
|        this|           t|
|       ebook|           e|
|          is|           i|
|         for|           f|
|         the|           t|
|         use|           u|
+------------+------------+
only showing top 20 rows



In [22]:
first_letter = first_letter.filter(first_letter.first_letter.rlike("[a-z]"))
first_letter.show(50)

+------------+------------+
|        word|first_letter|
+------------+------------+
|         the|           t|
|     project|           p|
|   gutenberg|           g|
|       ebook|           e|
|          of|           o|
|         the|           t|
|    complete|           c|
|       works|           w|
|          of|           o|
|     william|           w|
|shakespeare,|           s|
|          by|           b|
|     william|           w|
| shakespeare|           s|
|        this|           t|
|       ebook|           e|
|          is|           i|
|         for|           f|
|         the|           t|
|         use|           u|
|          of|           o|
|      anyone|           a|
|    anywhere|           a|
|          at|           a|
|          no|           n|
|        cost|           c|
|         and|           a|
|        with|           w|
|      almost|           a|
|          no|           n|
|restrictions|           r|
| whatsoever.|           w|
|         you|      

In [24]:
# groupby first letter and count word
grouped = first_letter.groupBy("first_letter").count()
grouped.show(26)

+------------+------+
|first_letter| count|
+------------+------+
|           l| 29569|
|           x|    14|
|           g| 20782|
|           m| 55676|
|           f| 36814|
|           n| 26759|
|           k|  9418|
|           v|  5728|
|           e| 18697|
|           o| 43494|
|           h| 60563|
|           z|    71|
|           p| 27759|
|           d| 29713|
|           w| 59597|
|           y| 25855|
|           c| 34567|
|           u|  9170|
|           i| 62167|
|           q|  2377|
|           j|  3339|
|           b| 45455|
|           a| 84836|
|           r| 14265|
|           t|123602|
|           s| 65705|
+------------+------+

